In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup,NavigableString, Tag
import time
import requests
import re
import csv
from selenium.webdriver.common.by import By
import pandas as pd
import os

headers = {'content-type': 'text/html; charset=UTF-8','user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}

driver = webdriver.Chrome()
driver.get("https://www.money101.com.tw/%E4%BF%A1%E7%94%A8%E5%8D%A1/%E7%B2%BE%E9%81%B8%E6%8E%A8%E8%96%A6")

time.sleep(5) #暫停

#soup = BeautifulSoup(driver.page_source, 'html.parser')
#print(soup.prettify())  #輸出排版後的HTML內容

button= driver.find_element(By.XPATH, "/html/body/div[1]/div/nav/div[1]/div[2]/ul/div[3]/li/label") #
if button.is_displayed() and button.is_enabled():
    button.click()
#button.click()
time.sleep(2)
        
soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀

result = soup.select('html body div:nth-of-type(1) div nav div:nth-of-type(1) div:nth-of-type(2) ul div:nth-of-type(3) li div div')
#print(result)
ranking_link = []
ranking_title = []
for item in result:
    link = item.find('a')
    if link:
        ranking_link.extend(['https://www.money101.com.tw' + a.get('href') for a in item.find_all('a')])
    span = item.find('span')
    if span:
        rank = span.text
        ranking_title.append(rank)
        
ranking_title.pop(0)
ranking_title.pop(0)
ranking_title.pop(0)
ranking_title.pop(0)
ranking_title.pop(0)

ranking_link.pop(0)
ranking_link.pop(0)
ranking_link.pop(0)

#print(ranking_link)
#print(ranking_title)
#print(len(ranking_link))
#print(len(ranking_title))


for k in range(len(ranking_link)-1):
    
    driver = webdriver.Chrome()
    driver.get(ranking_link[k+1])
    time.sleep(5) #暫停
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #print(soup.prettify())  #輸出排版後的HTML內容
    
    
    card_name = []
    card_link = []
    cards = soup.find_all('div',class_="flex items-center justify-between")
    for card in cards:
        title = card.find('h2',class_='pr-1')
        link = card.find('a').get('href')
        card_name.append(title.text)
        print('☆'+title.text)
        print('https://www.money101.com.tw'+link+'\n')
        card_link.append('https://www.money101.com.tw'+link)
        
    #單卡
    #最低年齡要求
    age_request = []
    #最低年薪要求
    salary_request = []
    #國外刷卡手續費
    fee = []
    #年費
    annual_fee = []
    #循環信用利率
    rate = []
    #信用卡發卡機構
    agency = []
    #card_name = []
    card_list2 = ['最低年齡要求','最低年薪要求','國外刷卡手續費','年費','循環信用利率','信用卡發卡機構']
        
    for j in range(len(card_link)):
    
        driver.get(card_link[j]) 
        
        time.sleep(5) #暫停
        
        soup = BeautifulSoup(driver.page_source, 'html.parser') #前往第一張卡的網頁
        
        print(card_name[j])
        
        result1 = soup.find_all('div', class_="box-border inline-block w-2/5 pr-2 align-top")#標題        
        card_list = []    #每張卡的標題（不一定是六個）    
        for result6 in result1:    
            card_list.append(result6.text)
            print(result6.text)
        
        result2 = soup.find_all('div', class_="inline-block w-1/2") #內文
        n=0
        for result in result2:
            if card_list[n] == "最低年齡要求":
                age_request.append(result.text) 
            elif card_list[n] == "最低年薪要求":
                salary_request.append(result.text)
            elif card_list[n] == "年費":
                fee.append(result.text)
            elif card_list[n] == "國外刷卡手續費":
                annual_fee.append(result.text)
            elif card_list[n] == "循環信用利率":
                rate.append(result.text)
            elif card_list[n] == "信用卡發卡機構":
                agency.append(result.text)       
            print(card_list[n]+":"+result.text+'\n')
            n+=1
            
        if len(age_request) != j+1:
            age_request.append('不適用')
        if len(salary_request) != j+1:
            salary_request.append('不適用')
        if len(fee) != j+1:
            fee.append('無')
        if len(annual_fee) != j+1:
            annual_fee.append('不適用')
        if len(rate) != j+1:
            rate.append('不適用')
        if len(agency) != j+1:
            agency.append('不適用')
            
        
        
        
        #table
        #可能沒有
        table = soup.find('div', class_='min-w-0 flex-1 md:ml-2')
        tables = table.find_all('table', style = 'width:100%') #如果有table
        title_list=[] #優惠csv
        text_list =[] #優惠csv
        if tables and ranking_title[k+1]== '台新銀行 台新街口聯名卡':
            title_list.append('活動一')
            title_list.append('活動二')
        #print(table)
        if tables:
            for table in tables:
                if table != None  and ranking_title[k+1]== '滙豐銀行 匯鑽卡':
                    results = table.find_all('tr')
                    i=0
                    header =[]
                    a = []
                    #b = []
                    for result in results:
                        r = result.find_all('td')
                        if i == 0:
                            for r in result:
                                #print(r.text)
                                header.append(r.text)
                            #print(i)
                        elif i ==1:
                            for r in result:
                                #print(r.text)
                                a.append(r.text)
                            #print(i)
                    #    elif i ==2:
                    #        for r in result:
                                #print(r.text)
                    #            b.append(r.text)
                            #print(i)
                        i=i+1
                    
                    for i in header:
                        if ' ' in header:
                            header.remove(' ')
                            
                    for i in a:
                        if ' ' in a:
                            a.remove(' ')
                            
                    #for i in b:
                    #    if ' ' in b:
                    #        b.remove(' ')
                    #b[0] = b[0][0:4]
                    print(header)
                    print(a)
                    #print(b)
                    #d = [a]
                    #df = pd.DataFrame(d, columns = header)
                    #print(df)
                    title_list.append(header[1]+'的'+a[0])
                    title_list.append(header[2]+'的'+a[0])
                    text_list.append(a[1])
                    text_list.append(a[2])
                    print(header[1]+'的'+a[0]+': '+a[1])
                    print(header[2]+'的'+a[0]+': '+a[2])
                #   print(header[1]+'的'+b[0]+': '+b[1])
                #   print(header[2]+'的'+b[0]+': '+b[2])
                elif table != None  and ranking_title[k+1]== '台新銀行 台新街口聯名卡':
                    table_header = []
                    table_text = []
                    trs = table.find_all('tr')
                    for tr in trs:
                        ths = tr.find_all('th')
                        tds = tr.find_all('td')
                        if ths !=None:
                            i = 0
                            for th in ths:
                                if i == 0:
                                    table_header.append(th.text)
                                elif i == 1:
                                    table_text.append(th.text)
                                print(th.text)
                                i+=1
                        else:
                            i = 0
                            for th in ths:
                                if i == 0:
                                    table_header.append(td.text)
                                elif i == 1:
                                    table_text.append(td.text)
                                print(td.text)
                                i+=1
                    texts=[]
                    for i in  range(len(table_header)):
                        texts.append(table_header[i]+':'+table_text[i])
                    t = '\n'.join(texts)
                    text_list.append(t)
                
                

        #聯邦銀行 吉鶴卡
        results = soup.find_all('div', class_='min-w-0 flex-1 md:ml-2') #產品特色、注意事項
        for result in results:
            print(result.find('p').text) #”產品特色“ “注意事項”
            result1 = result.find('ul', class_='-my-1') #產品特色大框、注意事項大框
            result2 = result1.find_all('li', class_='my-1 flex gap-1 font-small') #產品特色 勾 注意事項 橫槓
            for re in result2:
                text1 = re.find('div', class_ = 'rich-text w-11/12 md:w-full')
                table = text1.find('table', style = 'width:100%') #如果有table
                if table != None:
                    result3 = text1.find_all('li')
                    for res in result3:
                        print(res.text+'\n')
                        text_list.append(res.text)
                        title_list.append(result.find('p').text)
                else:
                    print(text1.text+'\n')
                    text_list.append(text1.text)
                    title_list.append(result.find('p').text)
        #print(text_list)
        
        result = soup.find('div', class_="px-2 md:px-0")
        result2 = result.find_all('span', class_="font-small font-bold")
        #print(result2)
        #title_list=[] #小標（消費類型）
        for result in result2:
            title_list.append(result.text)
            #print(result.text+'\n')
        
        for i in range(len(title_list)):
            print(title_list[i])
        
        result = soup.find('div', class_="px-2 md:px-0")
        result2 = result.find_all('div', class_="flex-1 pl-2 text-right")
        #print(result2)
        text=[] #內文(現金回饋)
        
        for result in result2:
            text.append(result.text)
            #print(result.text+'\n')
            
        text.pop(0)
        for i in range(len(text)):
            print(text[i])
            text_list.append(text[i])
            
        card_dict1={
            "標題":title_list,
            "詳情":text_list
        }
        
        df = pd.DataFrame(card_dict1)
        folder_name = 'money101'+ranking_title[k+1]
        path = "/Users/takoyaki/Downloads/專題爬蟲/money101csv"  
    
        folder_path = os.path.join(path, folder_name)
        if not os.path.isdir(folder_path):
            os.makedirs(folder_path, mode=0o777)
        path1 = str(path+'/'+folder_name+"/")
        #path='/Users/takoyaki/Downloads/專題爬蟲/money101csv/精選推薦卡csv/'
        df.to_csv(path1+"money101"+card_name[j]+"優惠.csv", index=False) #, index=False
        
    card_dict = {
        '卡片名稱': card_name[k+1],
        #card_list2[0]:age_request,
        #card_list2[1]:salary_request,
        card_list2[2]:fee,
        #card_list2[3]:annual_fee,
        card_list2[4]:rate
        #card_list2[5]:agency
    }    
    
    
    df = pd.DataFrame(card_dict)
    folder_name = 'money101'+ranking_title[k+1]
    path = "/Users/takoyaki/Downloads/專題爬蟲/money101csv"  

    folder_path = os.path.join(path, folder_name)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path, mode=0o777)
    path0 = str(path+'/'+folder_name+"/")
    #path1 = '/Users/takoyaki/Downloads/專題爬蟲/money101csv/精選推薦卡csv/' 
    df.to_csv(path0+"money101"+ranking_title[k+1]+"基本資訊"+'.csv', index=False)
    
driver.close() 